### scroll

In [ ]:
%%script node

const btnScroll = document.querySelector('.btn--scroll-to');
const sec1 = document.querySelector('#section--1');

btnScroll.addEventListener('click', (e)=>{
  //client stored in document element
  console.log(`client viewport height`, document.documentElement.clientHeight);
  console.log(`button coords`, e.target.getBoundingClientRect());
  //boundingClientRect coords relative to viewport
  const targCoords = sec1.getBoundingClientRect();
  console.log(`section coords`, targCoords);

  // //old way
  // window.scrollTo(
  //   //takes parameters x, y
  //   //or object with x, y & behavior
  //   {
  //     left: targCoords.left + scrollX, // adjusting to current scroll position
  //     top: targCoords.top + scrollY,
  //     // scroll behavior
  //     behavior: 'smooth'
  //   }
    
  //   //or total
  //   //targCoords
  //   )

  sec1.scrollIntoView({behavior: 'smooth'});

})

### events

#### onEvent properties


In [ ]:
%%script node

const h1 = document.querySelector('h1');

h1.onmouseenter = (e)=>{
    alert(`hovered on h1`);
}

// reassigning onEvent properties overrides previous function

#### addEventListener, removeEventListener
* allows adding multiple functions
* allows removing functions

do perform a function only once

In [ ]:
%%script node

const h1 = document.querySelector('h1');

const alertOnce = (e)=>{
    alert(`hovered on h1`);
    h1.removeEventListener('mouseenter', alertOnce)
}

h1.addEventListener('mouseenter', alertOnce)

perform a task after certain time

In [ ]:
%%script node

setTimeout(()=>h1.removeEventListener('mouseenter', alertOnce), 3000)

#### event propagation phases
* capturing at document root at the top
* move down to target at bottom
* back up to document root again through bubbling phase

event bubbling phase  
events, while bubbling, can trigger tasks in parent elements listening to same events 

stopEventPropagation  
prevents event from propagation from calling element /bubbling to parent element

In [6]:
%%script node

// event bubbling
const switchColor = function(e){
  
    const randInt = (min, max) => Math.floor(Math.random()*(max-min+1)+min);
    
    const randomColor = `rgb(${randInt(0,255)},${randInt(0,255)},${randInt(0,255)})`
    console.log(randomColor);
  
    this.style.backgroundColor = randomColor;
  
    //currentTarget element the event is currently propagation through
    //same as this 
    console.log(e.currentTarget===e.target, this);
      
  
    if(this==e.target){
        // stops event from bubbling once it reaches first parent
        e.stopPropagation();
    }
    
  }
  
  const trg = document.querySelector('.nav__link');
  const trgParent = document.querySelector('.nav__links');
  const trgGParent = document.querySelector('.nav');
  
  trg.addEventListener('click', switchColor)
  trgParent.addEventListener('click', switchColor)
  // to listen to child element events in capture phase add true
  trgGParent.addEventListener('click', switchColor, true)

rgb(169,29,226)


some events don't have capturing and bubbling phases

event delegation  
listening to event from parent instead of many target children

In [ ]:
%%script node

// add event listener to common parent element
document.querySelector('.nav__links').addEventListener('click', function(e){
    //prevents normal html-defined event-handler from triggering
    e.preventDefault();
    console.log(e.target === this);
    if(e.target.matches('.nav__link')){
      const id = e.target.getAttribute('href');
      document.querySelector(id).scrollIntoView({behavior: 'smooth'});
    }
    
})

In [ ]:
%%script node

const tabs = document.querySelectorAll('.operations__tab');
const tabsContain = document.querySelector('.operations__tab-container');
const tabsContent = document.querySelectorAll('.operations__content');

tabsContain.addEventListener('click', function(e){
  const clicked = e.target.closest('.operations__tab');

  // guard clause 
  if (!clicked) {
    console.log(e.target, clicked);
    return;
  }

  console.log(clicked);
  tabs.forEach(t=>t.classList.remove('operations__tab--active'))
  clicked.classList.add('operations__tab--active');

  const activNum = clicked.dataset.tab
  console.log(activNum);
  
  tabsContent.forEach(tc=>tc.classList.remove('operations__content--active'))

  document.querySelector(`.operations__content--${activNum}`).classList.add('operations__content--active');

})

intersection observer api event

In [ ]:
%%script node

const navStick = function (entries){
    entries.forEach(entry=>{
        console.log(`intersection ratio:`, entry.intersectionRatio);        
    })
    //const [entry] = entries;
    let intersection = entries[0].intersectionRatio;
    if(intersection==0){
        // const entry = entries[0];
        nav.classList.add('sticky')
        console.log(`added sticky`);
      } else {
        nav.classList.remove('sticky');
        console.log(`removed sticky`);
      }
  
  }
  
  // intersectionobservers are more efficient than scroll event listener
  const observer = new IntersectionObserver(
    //callback function
    navStick, 
    //observer options object
    {
      // root null means viewport
      root: null,
      // amount of intersection of observed element for creating entry
      theshold: [
        // when 0% of header is in viewport
        0,
        // when 100% of header is in viewport
        1
      ],
      // for margin before intersecting
      rootMargin: `-${nav.getBoundingClientRect().height}px`
    }
    );
  
  observer.observe(targ);

intersection observer entry.target

unobserve

In [ ]:
%%script node

const sects = document.querySelectorAll('.section');

const rvlSect = function(entries, observer) {
  const [entry]= entries;
  console.log(entries);

  if (!entry.isIntersecting) return;

  // target of observer is current
  console.log(entry.target);
  entry.target.classList.remove('section--hidden');

  // unobserver after finishing task
  observer.unobserve(entry.target);
}

const sectObs = new IntersectionObserver(
  rvlSect, 
  {
    root: null,
    threshold: [0.2]
  });

sects.forEach(sect=>{
  sect.classList.add('section--hidden');
  sectObs.observe(sect);
})

lazy-loading

In [ ]:
%%script node

// select element with given attribute
const imgLazies = document.querySelectorAll('img[data-src]');

const loadImg = function(entries, imgObs){
  const [entry] = entries;

  if (!entry.isIntersecting) return;
  const imgCur = entry.target;
  imgCur.src = entry.target.dataset.src;
  // wait for image to load
  imgCur.addEventListener('load', ()=>{
    imgCur.classList.remove('lazy-img');
    console.log(`image loaded`);
  })
  imgObs.unobserve(entry.target);
}

const imgObs = new IntersectionObserver(
  loadImg,
  {
    root: null,
    threshold: 0
  });

  imgLazies.forEach(img=>imgObs.observe(img));

slider with css transform

In [ ]:
const slides = document.querySelectorAll('.slide');

slides.forEach((s,i)=>{
  s.style.transform = `translateX(${100*i}%)`
})

let slideCur = 0;
const maxSlide = slides.length-1;

const goToSlide = function(dir){
  if(dir===btnLft||dir==='ArrowLeft'){
    if(slideCur===0){
      slideCur=maxSlide;
    } else {
      slideCur--;
    }

  } else if(dir===btnRgt||dir==='ArrowRight'){
    if(slideCur===maxSlide){
      slideCur=0;
    } else {
      slideCur++;
    }
  
  } else if(dir.matches('.dots__dot')){
    dotContain.childNodes.forEach(d=>{d.classList.remove('dots__dot--active')})
    dir.classList.add('dots__dot--active');
    slideCur=dir.dataset.slide;
  }

  slides.forEach((s,i)=>{
    s.style.transform = `translateX(${100*(i-slideCur)}%)`;
    console.log(i,slideCur,maxSlide,dir);
  });
}

const slider = document.querySelector('.slider');
const btnLft = document.querySelector('.slider__btn--left');
const btnRgt = document.querySelector('.slider__btn--right');

slider.addEventListener('click', function(e){
  goToSlide(e.target);
})

document.addEventListener('keydown', function(e){
  goToSlide(e.key)
})

goToSlide(slides[0]);

const dotContain = document.querySelector('.dots');

const createDots = function(){
  slides.forEach(function(_, i) {
    dotContain.insertAdjacentHTML(
      'beforeend', //option for inserting html before end
      `<button class="dots__dot" data-slide="${i}"></button>`
    );
  })
}

createDots();

DOMContentLoad event  
just HTML and javascript need to be loaded, not images & css

In [ ]:
%%script node

document.addEventListener('DOMContentLoaded', function(e){
    console.log('html, javascript loaded');
    
})

JQuery documentready function

In [ ]:
%%script node

document.ready

Load event  
when all html, css, and images load

In [ ]:
%%script node

window.addEventListener('load', function(e){
    console.log(`page fully loaded`);
    
})

unload event  
before user leaves page

In [ ]:
%%script node

window.addEventListener('beforeunload', function(e){
    e.preventDefault();
    console.log(`page closing`);
    // asks confirmation
    e.returnValue = '';
  })

regular
* parsing html: html converted to dom tree
* * when reaches script tag, javascript fetched & executed, before html parsing resumes
* * after html parsing finishes, domcontentloaded event fires

async

In [ ]:
<!-- script fetched while html is being parsed -->
<!-- so faster -->
<script async src=""></script>
<!-- script execution happens synchronously becasue javascript, 
pausing html parsing -->
<!-- domcontentloaded event may happen before script fetches and executes -->
<!-- scripts may not be executed in the order they are declared -->

defer

In [ ]:
<!-- script fetched while html is being parsed -->
<script defer src=""></script>
<!-- script execution happens synchronously after html parsed -->